In [5]:
import pandas as pd
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import os
from openai import OpenAI
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
api_key=os.getenv("API_KEY")
endpoint=os.getenv("ENDPOINT")

chat_client = OpenAI(api_key=api_key, 
                    base_url=endpoint,
                    default_headers={
        "HTTP-Referer": "http://localhost",
        "X-Title": "Dandori RAG Prototype"
    })

In [7]:
def load_data() -> list[dict]:
    data = pd.read_csv("all_courses.csv")

    data.drop(columns=["Unnamed: 0"], inplace=True)

    records = data.to_dict("records")

    return records

In [8]:
def chunker(records:list[dict]):
    chunks = []

    for i, row in enumerate(records):
        text = "\n".join(f"{k.title()}: {v}" for k, v in row.items())

        chunk = {
            "id": f"course_{i}",
            "text": text,
            "metadata": {
                "title": row.get("title"),
                "instructor": row.get("instructor"),
                "location": row.get("location"),
                "course_type": row.get("course_type"),
                "cost": row.get("cost"),
                "learning_objectives": row.get("learning_objectives"),
                "provided_materials": row.get("provided_materials"),
                "skills": row.get("skills_developed"),
            }
        }

        chunks.append(chunk)

    return chunks

In [9]:
def embed_data():
    pass

In [10]:
data: list[dict] = load_data()

In [11]:
chunks = chunker(data)

In [12]:
chunks[0]

{'id': 'course_0',
 'text': "Title: Cornish Pasty Poetry & Patisserie\nInstructor: Chef Rhubarb Mince\nLocation: Cornwall\nCourse_Type: Traditional Skills\nCost: 85.0\nLearning_Objectives: Master the art of crafting the perfect Cornish pasty, Learn to create deliciously whimsical pasty fillings, Discover how to weave poetry into pastry making, Understand the historical significance of pasty making, Develop culinary creativity and presentation skills\nProvided_Materials: nan\nSkills_Developed: Cooking, Creative Writing, Cultural History, Presentation, Creative Thinking\nDescription: Welcome to the Cornish Pasty Poetry & Patisserie class, where culinary arts meet lyrical expression in the heart of Cornwall. Under the guidance of Chef Rhubarb Mince, a master pastry chef with a flair for verse, you'll embark on a delightful journey of crafting perfect Cornish pasties while spicing them up with poetic charm. Students will start by getting hands-on with the basics of pasty dough making, lear

In [13]:
client = chromadb.Client()

In [14]:
collection = client.get_or_create_collection(
    name = "courses"
)

In [15]:
collection.add(
    ids=[i.get('id') for i in chunks] ,
    documents=[i.get('text') for i in chunks],
    metadatas=[i.get('metadata') for i in chunks],
)

In [16]:
results = collection.query(
    query_texts=["Pastry and Magic"], # Chroma will embed this for you
    n_results=5 # how many results to return
)

for i in range(5):
    print(results["documents"][0][i])
    print(results["distances"][0][i])

Title: Enchanted Pasty Crafting
Instructor: Chef Basil Parsnip
Location: Lake District
Course_Type: Culinary Arts
Cost: 95.0
Learning_Objectives: Master the art of making the perfect buttery pastry from scratch, Explore the enchanting history and myths surrounding the humble pasty, Learn to incorporate whimsical fillings inspired by local Lake District flora, Develop unique folding techniques with personalized pasty crest designs, Discover tips for creating magical flavor combinations that surprise and delight
Provided_Materials: nan
Skills_Developed: Baking, Creative Cooking, Local Cuisine, Herbal Knowledge, Artistic Presentation
Description: Join us in the heart of the picturesque Lake District for 'Enchanted Pasty Crafting', a culinary escapade led by the charming Chef Basil Parsnip. This class is perfect for adults looking to rekindle their joy through the delightful and slightly whimsical world of pasty creation. Students will embark on a flavorful journey, learning to create the 

In [18]:
query = "which courses have to do with myths"
top_results = 1

embedded_query = f"""{collection.query(
    query_texts=[query], # Chroma will embed this for you
    n_results=top_results 
)['documents']}\n 
{query}"""

response = chat_client.chat.completions.create(
    model="google/gemini-2.0-flash-001",
    messages=[
        {"role": "user", "content": embedded_query}
    ],
    temperature=0.4,
    max_tokens=512
)
print(response.choices[0].message.content)

Based on the provided course descriptions, the course that has to do with myths is:

*   **Highland Gnome Gathering with Gertie**: This course involves storytelling skills through whimsical folklore and shares captivating tales of Highland folklore.
